In [1]:
import os
os.chdir('..')

In [2]:
import pandas as pd
from tabulate import tabulate
import numpy as np
import itertools

### Configs through combinations for prerun and main run

In [3]:
alpha=0.8

beta=0.00000001

walker_wl=4

overlap_wl=0.25

seed_hist=501

seed_run=1

num_interactions=5000

replica_exchange_steps=20

intervals_main=16

iterations=1000

time_limit=10000

histogram_scale=1

qubit_specific_noise=0

num_loops = 1000

num_walker_prerun = 512

num_intervals_prerun = 64

error_mean = 0

error_variance = 0

In [4]:
prob_x_err = [0.03, 0.05, 0.06, 0.08]
prob_y_err = [0.03, 0.05, 0.06, 0.08]
prob_z_err = [0.03, 0.05, 0.06, 0.08]

x = [4, 6]
y = [4, 6]

xh_err = [0, 1]
xv_err = [0, 1]
zh_err = [0, 1]
zv_err = [0, 1]

# Combine all lists into a single list of lists
all_lists = [
    prob_x_err,
    prob_y_err,
    prob_z_err,
    x,
    y,
    xh_err,
    xv_err,
    zh_err,
    zv_err
]

# Use itertools.product to get all combinations
combinations = list(itertools.product(*all_lists))


In [7]:
# Use itertools.product to get all combinations
combinations = list(itertools.product(*all_lists))

# Create a DataFrame from the combinations
df_combinations = pd.DataFrame(combinations, columns=[
    'prob_x_err', 'prob_y_err', 'prob_z_err', 'x', 'y',
    'xh_err', 'xv_err', 'zh_err', 'zv_err'
])

df_combinations = df_combinations[df_combinations['x']==df_combinations['y']]
df_combinations = df_combinations[df_combinations[['prob_x_err', 'prob_y_err' ,'prob_z_err']].apply(lambda row: row.nunique() == 1, axis=1)]
df_combinations.reset_index(inplace=True, drop=True)

In [8]:
# Create a DataFrame for the parameters
params = pd.DataFrame({
    'ArrayTaskID': np.arange(1, len(df_combinations)+1),
    'alpha': [alpha] * len(df_combinations),
    'beta': [beta] * len(df_combinations),
    'walker_wl': [walker_wl] * len(df_combinations),
    'overlap_wl': [overlap_wl] * len(df_combinations),
    'seed_hist': [seed_hist] * len(df_combinations),
    'seed_run': range(seed_run, seed_run + len(df_combinations)),
    'num_interactions': [num_interactions] * len(df_combinations),
    'replica_exchange_steps': [replica_exchange_steps] * len(df_combinations),
    'intervals_main': [intervals_main] * len(df_combinations),
    'iterations': [iterations] * len(df_combinations),
    'time_limit': [time_limit] * len(df_combinations),
    'histogram_scale': [histogram_scale] * len(df_combinations),
    'qubit_specific_noise': [qubit_specific_noise] * len(df_combinations),
    'num_loops': [num_loops] * len(df_combinations),
    'num_walker_prerun': [num_walker_prerun] * len(df_combinations),
    'num_intervals_prerun': [num_intervals_prerun] * len(df_combinations),
    'error_mean': [error_mean] * len(df_combinations),
    'error_variance': [error_variance]*len(df_combinations)
})


#### Important Columns always need to be in the same order as right now

In [9]:
df_final = pd.concat([params, df_combinations], axis=1)

In [11]:
df_final.loc[df_final['x'] == 8, 'seed_hist'] = 1
df_final.loc[df_final['x'] == 8, 'num_interactions'] = 6000
df_final.loc[df_final['x'] == 8, 'intervals_main'] = 20

In [12]:
table_string = tabulate(df_final, headers="keys", tablefmt='plain', showindex=False)

with open('configs/eight_vertex.txt', 'w') as f:
    f.write(table_string)

### With for loops

In [4]:
rows = []
array_task = 1

for prob in [0.05]:
    for x in [4]:
        for y in [4]:
            if x==y:
                for xh_err in [0]:
                    for xv_err in [0]:
                        for zh_err in [0]:
                            for zv_err in [0]:
                                rows.append([
                                    array_task, alpha, beta, walker_wl, overlap_wl, seed_hist, seed_run, num_interactions, replica_exchange_steps,
                                    intervals_main, iterations, time_limit, num_loops, num_walker_prerun, num_intervals_prerun, prob, x, y,
                                    xh_err, xv_err, zh_err, zv_err, histogram_scale, qubit_specific_noise
                                ])

                                array_task += 1
                                seed_run += 1


In [5]:
df = pd.DataFrame(rows, columns=["ArrayTaskID", "alpha", "beta", "walker_wl", "overlap", "seed_hist", "seed_run",
                                "num_interactions", "replica_exchange_offset", "intervals_wl", "iterations",
                                "time_limit", "num_loops", "num_walker_prerun", "num_intervals_prerun", "probability",
                                "x", "y", "xh_err", "xv_err", "zh_err", "zv_err", "hist_scale", "qubit_specific_noise"])

In [8]:
# table_string = tabulate(df, headers="keys", tablefmt='plain', showindex=False)

# with open('configs/eight_vertex.txt', 'w') as f:
#     f.write(table_string)